# Planewaves in Homogeneous Media

The eigenmode source can also be used to launch [planewaves](https://en.wikipedia.org/wiki/Plane_wave) in homogeneous media. The dispersion relation for a planewave is ω=|$\vec{k}$|/$n$ where ω is the angular frequency of the planewave and $\vec{k}$ its wavevector; $n$ is the refractive index of the homogeneous medium. This example demonstrates launching planewaves in a uniform medium with $n$ of 1.5 at three rotation angles: 0°, 20°, and 40°. Bloch-periodic boundaries via the `k_point` are used and specified by the wavevector $\vec{k}$. PML boundaries are used only along the x-direction.

First, we'll load our necesarry modules:

In [10]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt

Next, we'll create a function we can call multiple times that runs the simulation for different rotation angles:

In [11]:
def run_sim(rot_angle=0):

    

    sx = 64
    sy = 32

    resolution = 25  # pixels/μm

    cell_size = mp.Vector3(sx, sy, 0)  # for 2D

    pml_thickness = 6

    pml_layers = [
    mp.PML(thickness=pml_thickness, direction=mp.Y),
    mp.PML(thickness=pml_thickness, direction=mp.X)
    ]


    fsrc = 1.0  # frequency of planewave (wavelength = 1/fsrc)

    n = 1  # refractive index of homogeneous material
    default_material = mp.Medium(index=n)

    k_point = mp.Vector3(y=-fsrc * n).rotate(mp.Vector3(z=1), rot_angle)

    geometry1 = [
        mp.Block(
            center=mp.Vector3(0, -sy/4),              # centered halfway down
            size=mp.Vector3(mp.inf, sy/2, mp.inf),    # fill y < 0
            material=mp.Medium(epsilon=2.56)
        )
    ]


    geometry2 = [
    mp.Cylinder(
        radius=1.0,
        height=mp.inf,                     # for 2D: extend infinitely in z
        center=mp.Vector3(0, 1),           # center at (0, 1)
        material=mp.Medium(epsilon=2.56)    # dielectric with ε=4 (or use mp.metal for PEC)
    )
    ]



    sources = [
        mp.EigenModeSource(
            src=mp.ContinuousSource(fsrc),
            center=mp.Vector3(0,sy/2 - pml_thickness + 2),
            size=mp.Vector3(x=sx),
            direction=mp.AUTOMATIC if rot_angle == 0 else mp.NO_DIRECTION,
            eig_kpoint=k_point,
            eig_band=1,
            eig_parity=mp.EVEN_Y + mp.ODD_Z if rot_angle == 0 else mp.ODD_Z,
            eig_match_freq=True,
        )
    ]

    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        geometry = geometry1 + geometry2,
        boundary_layers=pml_layers,
        sources=sources,
        k_point=k_point,
        default_material=default_material,
        symmetries=[mp.Mirror(mp.Y)] if rot_angle == 0 else [],
    )

    # Define positions of flux monitors (just inside the PML boundaries)
    flux_distance_from_boundary = pml_thickness + 0.1 # µm from boundary, adjust as needed

    # Top flux region (near top PML)
    flux_top = sim.add_flux(
        fsrc, 0, 1,
        mp.FluxRegion(center=mp.Vector3(0, 0.5*sy - flux_distance_from_boundary), size=mp.Vector3(sx, 0))
    )

    flux_1 = sim.add_flux(
    fsrc, 0, 1,
    mp.FluxRegion(center=mp.Vector3(0, sy/2 - (pml_thickness + 2.1)), size=mp.Vector3(sx, 0))
    )

    flux_2 = sim.add_flux(
    fsrc, 0, 1,
    mp.FluxRegion(center=mp.Vector3(0, 3), size=mp.Vector3(sx, 0))
    )

    flux_3 = sim.add_flux(
    fsrc, 0, 1,
    mp.FluxRegion(center=mp.Vector3(0, -0.1), size=mp.Vector3(sx, 0))
    )


    # Bottom flux region (near bottom PML, behind the source)
    flux_bottom = sim.add_flux(
        fsrc, 0, 1,
        mp.FluxRegion(center=mp.Vector3(0, -0.5*sy + flux_distance_from_boundary), size=mp.Vector3(sx, 0))
    )

    # Run your simulation
    sim.run(until=60)

    # Retrieve flux data
    flux_top_data = mp.get_fluxes(flux_top)
    flux_1_data = mp.get_fluxes(flux_1)
    flux_2_data = mp.get_fluxes(flux_2)
    flux_3_data = mp.get_fluxes(flux_3)
    flux_bottom_data = mp.get_fluxes(flux_bottom)

    print(f"Flux near top PML boundary: {flux_top_data[0]:.6f}")
    print(f"Flux at y = sy/2 - (pml_thickness + 2.1): {flux_1_data[0]:.6f}")
    print(f"Flux at y = 3: {flux_2_data[0]:.6f}")
    print(f"Flux at y = -0.1: {flux_3_data[0]:.6f}")
    print(f"Flux near bottom boundary: {flux_bottom_data[0]:.6f}")

    # Get field data
    ez = sim.get_array(center=mp.Vector3(), size=cell_size, component=mp.Ez)
    ex = sim.get_array(center=mp.Vector3(), size=cell_size, component=mp.Ex)
    ey = sim.get_array(center=mp.Vector3(), size=cell_size, component=mp.Ey)

    # Calculate |E| = sqrt(|Ex|^2 + |Ey|^2 + |Ez|^2)
    abs_E = np.sqrt(np.abs(ex)**2 + np.abs(ey)**2 + np.abs(ez)**2)

    # Plot
    plt.figure(figsize=(8, 4))
    plt.imshow(abs_E.transpose(), interpolation='spline36', cmap='viridis', origin='lower')
    plt.colorbar(label='|E|')
    plt.title('Magnitude of Electric Field |E|')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

    #plt.figure(dpi=100)
    #sim.plot2D(fields=mp.Ez)
    
    #plt.show()

Next we'll iterate over three rotation angles and plot their steady-state fields profiles. Residues of the backward-propagating waves due to the discretization are slightly visible.

In [ ]:
for rot_angle in np.radians([0.1, 20, 45, 60, 85]):
    run_sim(rot_angle)

-----------
Initializing structure...
time for choose_chunkdivision = 0.00279784 s
Working in 2D dimensions.
Computational cell is 64 x 32 x 0 with resolution 25
     block, center = (0,-8,0)
          size (1e+20,16,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2.56,2.56,2.56)
     cylinder, center = (0,1,0)
          radius 1, height 1e+20, axis (0, 0, 1)
          dielectric constant epsilon diagonal = (2.56,2.56,2.56)
time for set_epsilon = 3.38763 s
-----------
Meep: using complex fields.
MPB solved for frequency_1(0.00174533,-0.999998,0) = 1 after 308 iters
Meep progress: 2.52/60.0 = 4.2% done in 4.0s, 92.0s to go
on time step 126 (time=2.52), 0.0320151 s/step
Meep progress: 5.24/60.0 = 8.7% done in 8.1s, 84.2s to go
on time step 262 (time=5.24), 0.029588 s/step
Meep progress: 7.84/60.0 = 13.1% done in 12.1s, 80.3s to go
on time step 392 (time=7.84), 0.0308731 s/step
Meep progress: 10.38/60.0 = 17.3% done in 16.1s, 77.0s to go
o

Note that this example involves a `ContinuousSource` for the time profile. For a pulsed source, the oblique planewave is incident at a given angle for only a *single* frequency component of the source. This is a fundamental feature of FDTD simulations and not of Meep per se. Thus, to simulate an incident planewave at multiple angles for a given frequency ω, you will need to do separate simulations involving different values of $\vec{k}$ (`k_point`) since each set of ($\vec{k}$,ω) specifying the Bloch-periodic boundaries and the frequency of the source will produce a different angle of the planewave. For more details, refer to Section 4.5 ("Efficient Frequency-Angle Coverage") in [Chapter 4](https://arxiv.org/abs/1301.5366) ("Electromagnetic Wave Source Conditions") of [Advances in FDTD Computational Electrodynamics: Photonics and Nanotechnology](https://www.amazon.com/Advances-FDTD-Computational-Electrodynamics-Nanotechnology/dp/1608071707).